In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pip install -U langchain-community faiss-cpu langchain-openai tiktoken llama_index sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 54.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 50.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 66.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.0/237.0 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.1/225.1 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.7 MB/s eta 0:00:00
  Attempting uninstall: typing-

In [ ]:
import json
file_path= r'/content/drive/MyDrive/Colab Notebooks/Datasets/chunks_and_questions-ancient_history.json'
with open(file_path,'r') as file:
  data1=json.load(file)

In [ ]:
import json
file_path= r'/content/drive/MyDrive/Colab Notebooks/Datasets/chunks_and_questions-ancient_history.json'
with open(file_path,'r') as file:
  data_for_csv=json.load(file)

In [ ]:
data1[116]

{'chunk_id': 116,
 'context_chunk': 'Precious stones\nformed an important item of trade in articles which were eagerly sought for by\nthe Romans in the early centuries of the Christian era.\nChronology\n(\nBP\n)\n \n40 m years\nEmergence of the Indian subcontinent.\n2 m–1200\nThe last phase of the uplift of the Himalayas.\n(\nBC\n)\n \n1800\nEarly trace of gold in Karnataka.\n1000\nEarliest date of the Khetri copper belts.\n(\nAD\n)\n \n1 C\nDiscovery of the direction of the monsoon.',
 'chunk_questions': ['What was an important item of trade in ancient Rome?',
  'When did the Indian subcontinent emerge?',
  'When was the last phase of the uplift of the Himalayas?',
  'When were early traces of gold found in Karnataka?',
  'When was the Khetri copper belt dated to?',
  'When was the direction of the monsoon discovered?']}

 **Block of code to convert the json file to Document**

In [ ]:
from llama_index.schema import Document

# Assuming data1 is a list of dictionaries, and you want to create a Document for each item in the list
documents = []

for i in data1:
    doc_id = str(i.get("chunk_id", ""))
    text = i.get("context_chunk", "")
    metadata = {
        "chunk_id": i.get("chunk_id", ""),
        "context_chunk": i.get("context_chunk", ""),
        "chunk_questions": i.get("chunk_questions", [])
    }

    doc = Document(
        id_=doc_id,
        text=text,
        #metadata=metadata
    )
    documents.append(doc)


In [ ]:
documents[0]

Document(id_='0', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='1\nThe Significance of\nAncient Indian History\nThe study of ancient Indian history is important for several reasons. It tells us\nhow, when, and where people developed the earliest cultures in India, how they\nbegan undertaking agriculture and stock raising which made life secure and\nsettled. It shows how the ancient Indians discovered and utilized natural\nresources, and how they created the means for their livelihood. We get an idea of\nhow the ancient inhabitants made arrangements for food, shelter, and transport,\nand learn how they took to farming, spinning, weaving, metalworking, and the\nlike, how they cleared forests, founded villages, cities, and eventually large\nkingdoms.\nPeople are not considered civilized unless they know how to write. The\ndifferent forms of writing prevalent in India today are all derived from the\nancient scripts. Thi

In [ ]:
from llama_index import ServiceContext, VectorStoreIndex
from llama_index.embeddings import LangchainEmbedding
from langchain_community.embeddings import HuggingFaceEmbeddings
import numpy as np

#from sentence_transformers import SentenceTransformer
embed_model_2 = LangchainEmbedding(HuggingFaceEmbeddings(model_name="multi-qa-mpnet-base-dot-v1"))
#model = SentenceTransformer("sentence-transformers/multi-qa-mpnet-base-dot-v1")

service_context = ServiceContext.from_defaults(embed_model=embed_model_2, llm=None)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/8.66k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

LLM is explicitly disabled. Using MockLLM.


**Creating VectorStoreIndexes**

In [ ]:
index=VectorStoreIndex.from_documents(documents, service_context=service_context)

In [ ]:
import pickle
from llama_index import VectorStoreIndex

# index = VectorStoreIndex.from_documents(documents, service_context=service_context)

file_path = '/content/drive/MyDrive/Colab Notebooks/Embeddings/vector_index.pkl'

with open(file_path, 'wb') as file:
    pickle.dump(index, file)

with open(file_path, 'rb') as file:
    loaded_index = pickle.load(file)


In [ ]:
type(loaded_index)

llama_index.indices.vector_store.base.VectorStoreIndex

In [ ]:
retriever =index.as_retriever()

In [ ]:
retriever.retrieve("Why is the study of ancient Indian history important?")

[NodeWithScore(node=TextNode(id_='520cf83a-4009-4ecf-b6a7-0c4e4ab5bee0', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='0', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='6287b9cf9483c130de46b357dc4d041ee3a1725580c4133ff88d49cd1dbc442b'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='d0c190bc-e0ed-494d-b260-6c9c482436ec', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='87c469d509b3185884fcbc9e4ce1ac3abe42dcd73062d64a87670652c6927185')}, text='1\nThe Significance of\nAncient Indian History\nThe study of ancient Indian history is important for several reasons. It tells us\nhow, when, and where people developed the earliest cultures in India, how they\nbegan undertaking agriculture and stock raising which made life secure and\nsettled. It shows how the ancient Indians discovered and utilized natural\nresources, and how they created the means for the

In [ ]:

from llama_index import SimpleDirectoryReader

data = SimpleDirectoryReader('/content/drive/MyDrive/Colab Notebooks/Datasets').load_data()


In [ ]:
data[0]

Document(id_='8fafe503-9cb6-4666-8cb6-ffd100d63079', embedding=None, metadata={'file_path': '/content/drive/MyDrive/Colab Notebooks/Datasets/chunk_ids_questions.csv', 'file_name': 'chunk_ids_questions.csv', 'file_type': 'text/csv', 'file_size': 60432, 'creation_date': '2024-01-25', 'last_modified_date': '2024-01-25', 'last_accessed_date': '2024-01-25'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text="0, Why is the study of ancient Indian history important?\n0, What does ancient Indian history tell us about the earliest cultures in India?\n0, How did ancient Indians make their life secure and settled?\n0, In what ways did ancient Indians utilize natural resources?\n0, What activities were undertaken by ancient inhabitants for their livelihood?\n0, What we

In [ ]:
retriever = index.as_retriever(similarity_top_k=3)

In [ ]:
import pandas as pd
import llama_index
# from llama_index import NodeWithScore

queries= df["chunk_questions"].tolist()

def calculate_similarity(query):
    scores = retriever.retrieve(query)
    return scores

df['similarity_scores'] = None
# for i in range(len(queries)):
#   for j in range(len(queries[i])):
#     df['similarity_scores'] = calculate_similarity(queries[i][j])

for index, row in df.iterrows():
    questions = row['chunk_questions']
    scores_list = []

    for question in questions:
        scores = calculate_similarity(question)
        # scores= NodeWithScore.score
        for i in range(len(scores)):
          new_nodes=scores[i].score
          scores_list.append(new_nodes)
    df.at[index, 'similarity_scores'] = scores_list

print(df)


In [ ]:
nodes = retriever.retrieve("Why is the study of ancient Indian history important?")

In [ ]:
for key,chunk_id in nodes[2].node.relationships.items():
  print(chunk_id.node_id)
  break

79


In [ ]:
scores = [score["scores_list"] for score in scores]

In [ ]:
retriever.get_score()

In [ ]:
import pandas as pd
df_exploded = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Datasets/chunk_ids_questions.csv')

In [ ]:
retriever=index.as_retriever(similarity_top_k=3)

In [ ]:
df_exploded

,expected_chunk_id,chunk_questions
0,0,Why is the study of ancient Indian history imp...
1,0,What does ancient Indian history tell us about...
2,0,How did ancient Indians make their life secure...
3,0,In what ways did ancient Indians utilize natur...
4,0,What activities were undertaken by ancient inh...
...,...,...
909,116,When did the Indian subcontinent emerge?
910,116,When was the last phase of the uplift of the H...
911,116,When were early traces of gold found in Karnat...
912,116,When was the Khetri copper belt dated to?


In [ ]:
print(df_exploded.chunk_questions.unique())

['Why is the study of ancient Indian history important?'
 'What does ancient Indian history tell us about the earliest cultures in India?'
 'How did ancient Indians make their life secure and settled?'
 'In what ways did ancient Indians utilize natural resources?'
 'What activities were undertaken by ancient inhabitants for their livelihood?'
 'What were some industries practiced by ancient Indians?'
 'How did the ancient inhabitants establish villages, cities, and kingdoms?'
 'What is the significance of writing in determining civilization?'
 'Are the Indian scripts and languages used today derived from ancient ones?'
 'What is the origin of the different forms of writing in India?'
 'From where are the roots of the languages spoken in India today?'
 'What makes ancient Indian history interesting?'
 'Which races and tribes intermingled in early India?'
 'How did various ethnic groups contribute to Indian social system, art and architecture, language and literature?'
 'What is a remark

In [ ]:
"Why is the study of ancient Indian history important?" in df_exploded["chunk_questions"].values

True

In [ ]:
print(retriever.retrieve.__doc__)
print(retriever.retrieve.__annotations__)

Retrieve nodes given query.

        Args:
            str_or_query_bundle (QueryType): Either a query string or
                a QueryBundle object.

        
{'str_or_query_bundle': typing.Union[str, llama_index.schema.QueryBundle], 'return': typing.List[llama_index.schema.NodeWithScore]}


In [ ]:
results = []
for questions_list in df_exploded['chunk_questions']:
    questions_list = [questions_list]
    for question in questions_list:
        if type(question) == float:
            question = str(question)
        result = retriever.retrieve(question)
        results.append(result)

In [ ]:
results[1]

In [ ]:

sim_top_k_1 = []
sim_top_k_2 = []
sim_top_k_3 = []

for result_list in results:
    scores_1 = []
    scores_2 = []
    scores_3 = []

    for node_with_score in result_list:
        score = node_with_score.score

        scores_1.append(score)
        scores_2.append(score)
        scores_3.append(score)

    scores_1.sort(reverse=True)
    scores_2.sort(reverse=True)
    scores_3.sort(reverse=True)

    sim_top_k_1.extend(scores_1[:1])
    sim_top_k_2.extend(scores_2[1:2])
    sim_top_k_3.extend(scores_3[2:3])

print("Top 1 Scores:", sim_top_k_1)
print("Top 2 Scores:", sim_top_k_2)
print("Top 3 Scores:", sim_top_k_3)


Top 1 Scores: [0.8374294925785695, 0.7331695298190762, 0.583279847208563, 0.603641460662471, 0.5396947972870969, 0.5882531071543411, 0.5251836265732021, 0.6548367760408204, 0.7172957311161923, 0.6565882112375019, 0.6966282717551384, 0.7591491832001804, 0.6728794029338819, 0.6723968420864744, 0.6541012464193335, 0.584203566079985, 0.6137196572197863, 0.6282568699769346, 0.6482119262341199, 0.6468526641041931, 0.5930635820205294, 0.6091069963283895, 0.6363490261221288, 0.5134724990204387, 0.6518960383135964, 0.5912527295642203, 0.6603468511642993, 0.6524410972189525, 0.698185211457001, 0.5612640299808613, 0.5489798702800452, 0.6617334779268641, 0.7290085709100588, 0.6279478477219476, 0.5399070392711942, 0.6808321040114772, 0.66069740192284, 0.6614176794852388, 0.6451110036181334, 0.5282057267811943, 0.6338294154631713, 0.6739011356557343, 0.6258409616869244, 0.6697864123692493, 0.6429877303177878, 0.6137518788944469, 0.6637925701340591, 0.6487858925298858, 0.6165027283470845, 0.394431218

In [ ]:
df_exploded['sim_top_k_1'] = sim_top_k_1
df_exploded['sim_top_k_2'] = sim_top_k_2
df_exploded['sim_top_k_3'] = sim_top_k_3

print(df_exploded)


     expected_chunk_id                                    chunk_questions  \
0                    0  Why is the study of ancient Indian history imp...   
1                    0  What does ancient Indian history tell us about...   
2                    0  How did ancient Indians make their life secure...   
3                    0  In what ways did ancient Indians utilize natur...   
4                    0  What activities were undertaken by ancient inh...   
..                 ...                                                ...   
909                116           When did the Indian subcontinent emerge?   
910                116  When was the last phase of the uplift of the H...   
911                116  When were early traces of gold found in Karnat...   
912                116          When was the Khetri copper belt dated to?   
913                116  When was the direction of the monsoon discovered?   

     sim_top_k_1  sim_top_k_2  sim_top_k_3  
0       0.837429     0.683332 

In [ ]:
df_exploded

,expected_chunk_id,chunk_questions,sim_top_k_1,sim_top_k_2,sim_top_k_3
0,0,Why is the study of ancient Indian history imp...,0.837429,0.683332,0.667476
1,0,What does ancient Indian history tell us about...,0.733170,0.704626,0.677633
2,0,How did ancient Indians make their life secure...,0.583280,0.579808,0.556455
3,0,In what ways did ancient Indians utilize natur...,0.603641,0.545485,0.541369
4,0,What activities were undertaken by ancient inh...,0.539695,0.524085,0.519055
...,...,...,...,...,...
909,116,When did the Indian subcontinent emerge?,0.673535,0.672968,0.588288
910,116,When was the last phase of the uplift of the H...,0.594805,0.491004,0.488374
911,116,When were early traces of gold found in Karnat...,0.708806,0.660204,0.581590
912,116,When was the Khetri copper belt dated to?,0.595790,0.576736,0.542777


In [ ]:
df_exploded.to_csv('/content/drive/MyDrive/Colab Notebooks/Datasets/sim_scores_chunks.csv',index=False)

In [ ]:
import pandas as pd
df_exploded= pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Datasets/sim_scores_chunks.csv')

In [ ]:
results[0][0].node.relationships.items()

dict_items([(<NodeRelationship.SOURCE: '1'>, RelatedNodeInfo(node_id='0', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='6287b9cf9483c130de46b357dc4d041ee3a1725580c4133ff88d49cd1dbc442b')), (<NodeRelationship.NEXT: '3'>, RelatedNodeInfo(node_id='d0c190bc-e0ed-494d-b260-6c9c482436ec', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='87c469d509b3185884fcbc9e4ce1ac3abe42dcd73062d64a87670652c6927185'))])

In [ ]:
range(len(results[0]))

range(0, 3)

In [ ]:
for key, chunk_id_info in results[0][0].node.relationships.items():
  print(chunk_id_info.node_id)
  break

0


In [ ]:
results[0][0].node.relationships

{<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='0', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='6287b9cf9483c130de46b357dc4d041ee3a1725580c4133ff88d49cd1dbc442b'),
 <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='d0c190bc-e0ed-494d-b260-6c9c482436ec', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='87c469d509b3185884fcbc9e4ce1ac3abe42dcd73062d64a87670652c6927185')}

In [ ]:

top_1_id=[]
top_2_id=[]
top_3_id=[]

for result in results:
    for key, chunk_id_info in result[0].node.relationships.items():
      top_1_id.append(chunk_id_info.node_id)
      break
    for key, chunk_id_info in result[1].node.relationships.items():
      top_2_id.append(chunk_id_info.node_id)
      break
    for key, chunk_id_info in result[2].node.relationships.items():
      top_3_id.append(chunk_id_info.node_id)
      break

In [ ]:
len(top_1_id)

914

In [ ]:
df_exploded["top_1_chunk"] = top_1_id
df_exploded["top_2_chunk"] = top_2_id
df_exploded["top_3_chunk"] = top_3_id

In [ ]:
df_exploded

,expected_chunk_id,chunk_questions,sim_top_k_1,sim_top_k_2,sim_top_k_3,top_1_chunk,top_2_chunk,top_3_chunk
0,0,Why is the study of ancient Indian history imp...,0.837429,0.683332,0.667476,0,14,79
1,0,What does ancient Indian history tell us about...,0.733170,0.704626,0.677633,0,1,12
2,0,How did ancient Indians make their life secure...,0.583280,0.579808,0.556455,109,108,0
3,0,In what ways did ancient Indians utilize natur...,0.603641,0.545485,0.541369,109,0,3
4,0,What activities were undertaken by ancient inh...,0.539695,0.524085,0.519055,44,109,97
...,...,...,...,...,...,...,...,...
909,116,When did the Indian subcontinent emerge?,0.673535,0.672968,0.588288,89,90,91
910,116,When was the last phase of the uplift of the H...,0.594805,0.491004,0.488374,90,93,94
911,116,When were early traces of gold found in Karnat...,0.708806,0.660204,0.581590,114,115,116
912,116,When was the Khetri copper belt dated to?,0.595790,0.576736,0.542777,111,110,116


In [ ]:
df_exploded.to_csv('/content/drive/MyDrive/Colab Notebooks/Datasets/final_res.csv', index= False)

In [ ]:
import json
file_path= r'/content/drive/MyDrive/Colab Notebooks/Datasets/test/question_and_answers-ancient_history (2).json'
with open(file_path,'r') as file:
  data_doc=json.load(file)

In [ ]:
data_doc = pd.DataFrame(data_doc)

In [ ]:
data_doc

,chunk_question,answer,chunk_id,question_id
0,Why is the study of ancient Indian history imp...,The study of ancient Indian history is importa...,0,0
1,What is the origin of the different forms of w...,The origin of the different forms of writing i...,1,9
2,What are the two main elements that the Aryan ...,"Based on the given context, the two main eleme...",2,15
3,What linguistic influence can be observed in t...,"In the given context, it is observed that the ...",3,21
4,What was the countrywide name for the Indian s...,The countrywide name for the Indian subcontine...,4,28
...,...,...,...,...
115,What are some must-visit travel destinations t...,out of context,-,907
116,What are some key factors to consider when cho...,out of context,-,908
117,Can you share any tips for game streaming or c...,out of context,-,909
118,What are some appropriate questions to ask the...,out of context,-,910


In [ ]:
queries = data_doc['chunk_question'].tolist()

In [ ]:
retriever=index.as_retriever(similarity_top_k=3)

In [ ]:
results_new= []
for query in queries:
  result=retriever.retrieve(query)
  results_new.append(result)

In [ ]:
results_new[0]

[NodeWithScore(node=TextNode(id_='520cf83a-4009-4ecf-b6a7-0c4e4ab5bee0', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='0', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='6287b9cf9483c130de46b357dc4d041ee3a1725580c4133ff88d49cd1dbc442b'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='d0c190bc-e0ed-494d-b260-6c9c482436ec', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='87c469d509b3185884fcbc9e4ce1ac3abe42dcd73062d64a87670652c6927185')}, text='1\nThe Significance of\nAncient Indian History\nThe study of ancient Indian history is important for several reasons. It tells us\nhow, when, and where people developed the earliest cultures in India, how they\nbegan undertaking agriculture and stock raising which made life secure and\nsettled. It shows how the ancient Indians discovered and utilized natural\nresources, and how they created the means for the

In [ ]:

sim_top_1 = []
sim_top_2 = []
sim_top_3 = []

for result_list in results_new:
    scores_1 = []
    scores_2 = []
    scores_3 = []

    for node_with_score in result_list:
        score = node_with_score.score

        scores_1.append(score)
        scores_2.append(score)
        scores_3.append(score)

    scores_1.sort(reverse=True)
    scores_2.sort(reverse=True)
    scores_3.sort(reverse=True)

    sim_top_1.extend(scores_1[:1])
    sim_top_2.extend(scores_2[1:2])
    sim_top_3.extend(scores_3[2:3])

print("Top 1 Scores:", sim_top_1)
print("Top 2 Scores:", sim_top_2)
print("Top 3 Scores:", sim_top_3)


Top 1 Scores: [0.8374294925785695, 0.6565882112375019, 0.584203566079985, 0.6091069963283895, 0.698185211457001, 0.5282057267811943, 0.3944312183870033, 0.5854892162579325, 0.7101423811160202, 0.6535318973504307, 0.6530019408640678, 0.6294715640006905, 0.5434047022538371, 0.8332391931524504, 0.5318790052072699, 0.5033323302278747, 0.5134890237090417, 0.6872983308710319, 0.47376542947774075, 0.5996760073470255, 0.687380130579686, 0.7284516066374362, 0.6263187321011136, 0.5306137512105845, 0.6873101221105203, 0.5066587039243682, 0.7000038298488055, 0.7005576059280219, 0.6722515918042999, 0.646301847836532, 0.5870811919975678, 0.531520849594293, 0.5859863161493299, 0.5100251429337401, 0.6672013512402768, 0.6439721837783511, 0.6328135165813007, 0.6822249448666781, 0.6532534482420177, 0.6023394391345241, 0.7269028855310273, 0.7447156511247285, 0.6229540293402601, 0.6589380866237589, 0.6667358794657615, 0.7004417068990251, 0.6776686867237542, 0.6385504543580286, 0.7317821830117636, 0.6691293

In [ ]:
len(sim_top_1)

120

In [ ]:
import pandas as pd
df=pd.DataFrame(data_doc)
selected_columns = ["chunk_question", "chunk_id"]
df = data_doc[selected_columns]

In [ ]:
df

,chunk_question,chunk_id
0,Why is the study of ancient Indian history imp...,0
1,What is the origin of the different forms of w...,1
2,What are the two main elements that the Aryan ...,2
3,What linguistic influence can be observed in t...,3
4,What was the countrywide name for the Indian s...,4
...,...,...
115,What are some must-visit travel destinations t...,-
116,What are some key factors to consider when cho...,-
117,Can you share any tips for game streaming or c...,-
118,What are some appropriate questions to ask the...,-


In [ ]:
df['sim_top_1'] = sim_top_1
df['sim_top_2'] = sim_top_2
df['sim_top_3'] = sim_top_3

<ipython-input-92-4052370f9cfd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sim_top_1'] = sim_top_1


In [ ]:

top_1_chunk_id=[]
top_2_chunk_id=[]
top_3_chunk_id=[]

for result in results_new:
    for key, chunk_id_info in result[0].node.relationships.items():
      top_1_chunk_id.append(chunk_id_info.node_id)
      break
    for key, chunk_id_info in result[1].node.relationships.items():
      top_2_chunk_id.append(chunk_id_info.node_id)
      break
    for key, chunk_id_info in result[2].node.relationships.items():
      top_3_chunk_id.append(chunk_id_info.node_id)
      break

In [ ]:
top_1_chunk_id

['0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '6',
 '8',
 '9',
 '10',
 '10',
 '12',
 '0',
 '14',
 '15',
 '17',
 '18',
 '24',
 '20',
 '21',
 '0',
 '24',
 '24',
 '18',
 '27',
 '29',
 '30',
 '31',
 '32',
 '79',
 '33',
 '37',
 '38',
 '39',
 '40',
 '41',
 '41',
 '44',
 '45',
 '46',
 '46',
 '48',
 '46',
 '50',
 '51',
 '52',
 '53',
 '53',
 '56',
 '57',
 '57',
 '59',
 '60',
 '61',
 '62',
 '62',
 '64',
 '65',
 '67',
 '62',
 '24',
 '71',
 '72',
 '73',
 '74',
 '50',
 '76',
 '78',
 '88',
 '85',
 '82',
 '84',
 '85',
 '85',
 '37',
 '76',
 '89',
 '91',
 '92',
 '92',
 '94',
 '95',
 '96',
 '89',
 '100',
 '102',
 '104',
 '104',
 '106',
 '1',
 '109',
 '109',
 '2',
 '112',
 '113',
 '109',
 '116',
 '116',
 '7',
 '87',
 '29',
 '21',
 '65',
 '1',
 '24',
 '12',
 '25',
 '91',
 '67',
 '86',
 '21',
 '88',
 '40',
 '0',
 '47',
 '109',
 '65',
 '87',
 '21']

In [ ]:
df["top_1_chunk"]=top_1_chunk_id
df["top_2_chunk"]=top_2_chunk_id
df["top_3_chunk"]=top_3_chunk_id

In [ ]:
df

,chunk_question,chunk_id,sim_top_1,sim_top_2,sim_top_3,top_1_chunk,top_2_chunk,top_3_chunk
0,Why is the study of ancient Indian history imp...,0,0.837429,0.683332,0.667476,0,14,79
1,What is the origin of the different forms of w...,1,0.656588,0.610021,0.605884,1,51,56
2,What are the two main elements that the Aryan ...,2,0.584204,0.583348,0.569782,2,1,88
3,What linguistic influence can be observed in t...,3,0.609107,0.603722,0.556580,3,1,2
4,What was the countrywide name for the Indian s...,4,0.698185,0.681981,0.664277,4,89,5
...,...,...,...,...,...,...,...,...
115,What are some must-visit travel destinations t...,-,0.280669,0.278453,0.269350,47,93,71
116,What are some key factors to consider when cho...,-,0.242887,0.234347,0.228130,109,85,86
117,Can you share any tips for game streaming or c...,-,0.246466,0.234525,0.208974,65,88,98
118,What are some appropriate questions to ask the...,-,0.312795,0.259833,0.248745,87,31,21


In [ ]:
df.to_csv("/content/drive/MyDrive/Colab Notebooks/Datasets/new_res.csv", index=False)